In [1]:
import os
os.environ['USE_PYGEOS'] = '0'

from functions import prepare_folders, download_images_for_points, get_gvi_per_buffer, select_points_on_road_network, get_features_on_points, get_road_network_with_points, select_points_within_buffers
import geopandas as gpd
from datetime import timedelta
from time import time

In [2]:
file = "Test_multiple_home_locations.gpkg" # File with address points
layer = "Test_multiple_home_locations"
buffer_distance = 500 # buffer distance in meters
access_token = "MLY|6267906093323631|fba37c53726a386c951323ee5b9874bf" # Access token for mapillary

In [3]:
prepare_folders()

In [4]:
# Read the file as a GeoDataFrame
gdf_points = gpd.read_file(file, layer=layer) # The crs must use meters
gdf_points['buffer'] = gdf_points['geometry'].buffer(buffer_distance)

In [5]:
# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
gdf_points.to_crs(epsg=4326, inplace=True)

# Reproject the 'buffer' column to EPSG 4326
gdf_points['buffer'] = gdf_points['buffer'].to_crs(epsg=4326)

In [6]:
road = get_road_network_with_points(gdf_points['buffer'])

# Reproject the GeoDataFrame to WGS84 (EPSG:4326)
road.to_crs(epsg=4326, inplace=True)

In [7]:
road_points = select_points_on_road_network(road)

In [8]:
points = select_points_within_buffers(gdf_points, road_points)

In [9]:
# Get features for each point
features = get_features_on_points(points, access_token)

file_path = os.path.join("results", "points.gpkg")
features.to_file(file_path, driver="GPKG")

# Get the initial time
start_time = time()

# The number of workers must be adapted depending on the computer
gvi_per_point = download_images_for_points(features, access_token, 3)
# Get the final time
end_time = time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Format the elapsed time as "hh:mm:ss"
formatted_time = str(timedelta(seconds=elapsed_time))

print(f"Running time: {formatted_time}")

Processing images (Process 1): 100%|██████████| 11/11 [05:33<00:00, 30.29s/it]


Running time: 0:05:42.708556


In [10]:
gvi_buffer, points = get_gvi_per_buffer(gdf_points, gvi_per_point)

In [11]:
display(gvi_buffer)

,geometry,GVI,Point_Count
0,POINT (-2.17238 53.43243),0.167305,24


In [12]:
display(points)

# index has the UID of the point (or buffer)
# geometry_left has the coordinates of the sample point that is within the buffer
# GVI has the GVI value, NaN in case the image is missing
# is_panoramic indicates if the image is panoramic, None in case it is missing
# missing indicates if the images was usable (road centers were identified) or not

,index_right,level_1,geometry_left,GVI,is_panoramic,missing
0,0,0,POINT (-2.16857 53.43363),0.239231,False,False
1,0,1,POINT (-2.16738 53.43337),0.239231,False,False
2,0,2,POINT (-2.17795 53.42943),0.093304,False,False
3,0,3,POINT (-2.16753 53.43313),NaN,None,True
4,0,4,POINT (-2.16714 53.43382),0.239231,False,False
...,...,...,...,...,...,...
80,0,80,POINT (-2.16551 53.43113),NaN,None,True
81,0,81,POINT (-2.17777 53.43070),NaN,None,True
82,0,82,POINT (-2.17111 53.43585),0.239231,False,False
83,0,83,POINT (-2.16742 53.43158),NaN,None,True


In [13]:
file_path = os.path.join("results", "GVI-points.gpkg")
points.to_file(file_path, driver="GPKG")

file_path = os.path.join("results", "GVI-buffer.gpkg")
gvi_buffer.to_file(file_path, driver="GPKG")